In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from sklearn.model_selection import KFold
import xgboost as xgb
import gc

# 读取和准备数据
df_all_feats = pd.read_csv("df_all_feats.csv")
df_all = pd.read_csv("df_all.csv")

# 过滤数据
X3_all = df_all_feats.dropna(subset=["value"])
X3_all = X3_all[~X3_all["feature"].isin(["age", "age_cln", "age_cln2"])]

X3_all["feature_name"] = X3_all["feature"]
X3_all["feature"] = pd.factorize(X3_all["feature"])[0]

X3 = X3_all[X3_all["id"].isin(df_all[df_all["age"].notna()]["id"])]
X3_test = X3_all[X3_all["id"].isin(df_all[df_all["age"].isna()]["id"])]
X3_all_feature = (
    X3_all[["feature", "feature_name"]].drop_duplicates().sort_values("feature")
)

X3["id_num"] = pd.factorize(X3["id"])[0]
X3_test["id_num"] = pd.factorize(X3_test["id"])[0]

X3_id = pd.DataFrame({"id": X3["id"].unique(), "id_num": X3["id_num"].unique()})
X3_id = X3_id.merge(df_all[["id", "age"]], on="id").sort_values("id_num")
y3 = X3_id["age"].values

X3_test_id = pd.DataFrame(
    {"id": X3_test["id"].unique(), "id_num": X3_test["id_num"].unique()}
).sort_values("id_num")
y3_test = np.full(len(X3_test_id), np.nan)

common_feature = np.intersect1d(X3["feature"].unique(), X3_test["feature"].unique())

X3 = X3[X3["feature"].isin(common_feature)].dropna()
X3_sp = coo_matrix((X3["value"], (X3["id_num"], X3["feature"])))
X3_test = X3_test[X3_test["feature"].isin(common_feature)].dropna()
X3_test_sp = coo_matrix((X3_test["value"], (X3_test["id_num"], X3_test["feature"])))

dX3 = xgb.DMatrix(X3_sp, label=y3)
dX3_test = xgb.DMatrix(X3_test_sp)